In [0]:
from google.colab import drive
drive.mount("/content/drive/")

Drive already mounted at /content/drive/; to attempt to forcibly remount, call drive.mount("/content/drive/", force_remount=True).


In [0]:
cd /content/drive/My Drive/Datasets/extracted/flower kaggle/

/content/drive/My Drive/Datasets/extracted/flower kaggle


In [0]:
import os
print(os.listdir('flowers'))

['dandelion', 'rose', 'sunflower', 'tulip', 'daisy']


In [0]:
# Ignore  the warnings
import warnings
warnings.filterwarnings('always')
warnings.filterwarnings('ignore')

# data visualisation and manipulation
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from matplotlib import style
import seaborn as sns
 
#configure
# sets matplotlib to inline and displays graphs below the corressponding cell.
%matplotlib inline  
style.use('fivethirtyeight')
sns.set(style='whitegrid',color_codes=True)

#model selection
from sklearn.model_selection import train_test_split
# from sklearn.model_selection import KFold
# from sklearn.metrics import accuracy_score,precision_score,recall_score,confusion_matrix,roc_curve,roc_auc_score
# from sklearn.model_selection import GridSearchCV
from sklearn.preprocessing import LabelEncoder

#preprocess.
from keras.preprocessing.image import ImageDataGenerator

#dl libraraies
from keras import backend as K
from keras.models import Sequential
from keras.layers import Dense
from keras.optimizers import Adam,SGD,Adagrad,Adadelta,RMSprop
from keras.utils import to_categorical

# specifically for cnn
from keras.layers import Dropout, Flatten,Activation
from keras.layers import Conv2D, MaxPooling2D, BatchNormalization
 
import tensorflow as tf
import random as rn

# specifically for manipulating zipped images and getting numpy arrays of pixel values of images.
import cv2                  
import numpy as np  
from tqdm import tqdm
import os                   
from random import shuffle  
from PIL import Image

In [0]:
class_names = ['daisy','rose','sunflower','tulip']

In [0]:
X=[]
Z=[]
IMG_SIZE=224
FLOWER_DAISY_DIR='flowers/daisy'
FLOWER_SUNFLOWER_DIR='flowers/sunflower'
FLOWER_TULIP_DIR='flowers/tulip'
#FLOWER_DANDI_DIR='flowers/dandelion'
FLOWER_ROSE_DIR='flowers/rose'

In [0]:
def assign_label(img, flower_type):
    return flower_type

In [0]:
# from PIL import Image
# from resizeimage import resizeimage

def make_train_data(flower_type,DIR):
    for img in tqdm(os.listdir(DIR)):
        label=assign_label(img,flower_type)
        path = os.path.join(DIR,img)
        img = cv2.imread(path,cv2.IMREAD_COLOR)
        img = cv2.resize(img, (IMG_SIZE,IMG_SIZE))
        
        X.append(np.array(img))
        Z.append(str(label))
        
    

In [0]:
make_train_data(class_names[0],FLOWER_DAISY_DIR)
print(len(X))

100%|██████████| 769/769 [00:04<00:00, 184.39it/s]

769


In [0]:
make_train_data(class_names[1],FLOWER_ROSE_DIR)
print(len(X))

100%|██████████| 784/784 [00:04<00:00, 182.92it/s]

1553


In [0]:
make_train_data(class_names[2],FLOWER_SUNFLOWER_DIR)
print(len(X))

100%|██████████| 734/734 [00:04<00:00, 168.32it/s]

2287


In [0]:
make_train_data(class_names[3],FLOWER_TULIP_DIR)
print(len(X))

100%|██████████| 984/984 [00:05<00:00, 176.38it/s]

3271


In [0]:
print(len(X))

3271


In [0]:
# fig,ax=plt.subplots(5,2)
# fig.set_size_inches(15,15)
# for i in range(5):
#     for j in range (2):
#         l=rn.randint(0,len(Z))
#         ax[i,j].imshow(X[l])
#         ax[i,j].set_title('Flower: '+Z[l])
        
# plt.tight_layout()
        

In [0]:
le=LabelEncoder()
Y=le.fit_transform(Z)
Y=to_categorical(Y,4)
X=np.array(X)
X=X/255

In [0]:
x_train,x_test,y_train,y_test=train_test_split(X,Y,test_size=0.2,random_state=42)


In [0]:
x_train.shape

(2616, 224, 224, 3)

In [0]:
np.random.seed(42)
rn.seed(42)
tf.set_random_seed(42)

In [0]:
datagen = ImageDataGenerator(
        featurewise_center=False,  # set input mean to 0 over the dataset
        samplewise_center=False,  # set each sample mean to 0
        featurewise_std_normalization=False,  # divide inputs by std of the dataset
        samplewise_std_normalization=False,  # divide each input by its std
        zca_whitening=False,  # apply ZCA whitening
        rotation_range=10,  # randomly rotate images in the range (degrees, 0 to 180)
        zoom_range = 0.1, # Randomly zoom image 
        width_shift_range=0.2,  # randomly shift images horizontally (fraction of total width)
        height_shift_range=0.2,  # randomly shift images vertically (fraction of total height)
        horizontal_flip=True,  # randomly flip images
        vertical_flip=False)  # randomly flip images


datagen.fit(x_train)

In [0]:
cd /content/drive/My Drive/Datasets/deep-learning-models/

/content/drive/My Drive/Datasets/deep-learning-models


In [0]:
!ls

imagenet_utils.py  resnet50.py		     vgg16.py		      vgg19.py
inception_v3.py    vgg162.weights.best.hdf5  vgg16.weights.best.hdf5


## Transfer Learning res_net 50

In [0]:
from keras.applications.resnet50 import ResNet50
from keras.preprocessing import image
from keras.applications.resnet import preprocess_input
import numpy as np
from keras.layers import merge, Input
from keras.models import Model

In [0]:
image_input = Input(shape=(224 ,224, 3))

### Only Training with the last Layer and Rest layer will be freezed

In [0]:
resNet = ResNet50(input_tensor=image_input, include_top=True,weights='imagenet')
resNet.summary()

Model: "resnet50"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_3 (InputLayer)            (None, 224, 224, 3)  0                                            
__________________________________________________________________________________________________
conv1_pad (ZeroPadding2D)       (None, 230, 230, 3)  0           input_3[0][0]                    
__________________________________________________________________________________________________
conv1 (Conv2D)                  (None, 112, 112, 64) 9472        conv1_pad[0][0]                  
__________________________________________________________________________________________________
bn_conv1 (BatchNormalization)   (None, 112, 112, 64) 256         conv1[0][0]                      
___________________________________________________________________________________________

In [0]:
last_layer = resNet.get_layer('avg_pool')
#out = resNet.output
#print(last_layer.output_shape)
last_out = last_layer.output
#x = Flatten()(last_out)
# print(x)
out = Dense(4, activation='softmax', name='output_layer')(last_out)
edt_resNet = Model(image_input,out)
edt_resNet.summary()

(None, 2048)
Model: "model_2"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_3 (InputLayer)            (None, 224, 224, 3)  0                                            
__________________________________________________________________________________________________
conv1_pad (ZeroPadding2D)       (None, 230, 230, 3)  0           input_3[0][0]                    
__________________________________________________________________________________________________
conv1 (Conv2D)                  (None, 112, 112, 64) 9472        conv1_pad[0][0]                  
__________________________________________________________________________________________________
bn_conv1 (BatchNormalization)   (None, 112, 112, 64) 256         conv1[0][0]                      
_______________________________________________________________________________

In [0]:
for layer in edt_resNet.layers[:-1]:
	layer.trainable = False

In [0]:
edt_resNet.compile(loss='categorical_crossentropy',optimizer='adam',metrics=['accuracy'])


In [0]:
from keras.callbacks import ModelCheckpoint
filepath="resNet50.weights.best.hdf5"
checkpoint = ModelCheckpoint(filepath, monitor='val_acc', verbose=1, save_best_only=True, mode='max')
callbacks_list = [checkpoint]

In [0]:
History = edt_resNet.fit_generator(datagen.flow(x_train,y_train, batch_size=64),
                              epochs = 10, validation_data = (x_test,y_test),
                              verbose = 1, steps_per_epoch=x_train.shape[0] // 64,
                              callbacks=callbacks_list)

NameError: ignored

## Visualizing Predictions On the Validation Set

In [0]:
# getting predictions on val set.
pred=edt_resNet.predict(x_test)
pred_digits=np.argmax(pred,axis=1)

In [0]:
# now storing some properly as well as misclassified indexes'.
i=0
prop_class=[]
mis_class=[]

for i in range(len(y_test)):
    if(np.argmax(y_test[i])==pred_digits[i]):
        prop_class.append(i)
#     if(len(prop_class)==8):
#         break

i=0
for i in range(len(y_test)):
    if(not np.argmax(y_test[i])==pred_digits[i]):
        mis_class.append(i)
#     if(len(mis_class)==8):
#         break

In [0]:
print(len(prop_class))
print(len(mis_class))

In [0]:
plt.imshow(x_test[5])
#pred[]
print(class_names[np.argmax(pred[5])])